# Walmart Sales Forecasting - Advanced Models

## Overview
This notebook implements advanced forecasting models to improve upon baseline performance:
- Gradient boosting models (XGBoost, LightGBM)
- Time series specific models (Prophet)
- Hyperparameter optimization with Optuna
- Advanced validation strategies
- Model ensembling techniques

## Advanced Model Strategy
1. **Gradient Boosting**: XGBoost and LightGBM with hyperparameter tuning
2. **Time Series Models**: Prophet for trend and seasonality
3. **Hyperparameter Optimization**: Automated tuning with Optuna
4. **Cross-Validation**: Time series cross-validation with multiple folds
5. **Ensemble Methods**: Combining multiple models for better performance
6. **Feature Selection**: Advanced feature importance and selection techniques

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import pickle
import json
import os
import sys
from pathlib import Path
from tqdm import tqdm

# Advanced ML imports
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import VotingRegressor
from sklearn.feature_selection import SelectKBest, f_regression

# Hyperparameter optimization
import optuna
from optuna.samplers import TPESampler

# Time series models
try:
    from prophet import Prophet
    PROPHET_AVAILABLE = True
except ImportError:
    print("Prophet not available, skipping Prophet models")
    PROPHET_AVAILABLE = False

# Add project src to path
sys.path.append(str(Path('../src').resolve()))

# Import project modules
try:
    from data.data_loader import DataLoader
    from utils.config import PROCESSED_DATA_DIR, MODEL_CONFIG, METRICS_CONFIG
    from utils.logger import get_project_logger
    logger = get_project_logger('advanced_models')
    USE_PROJECT_MODULES = True
except ImportError:
    print("Project modules not available, using direct imports")
    USE_PROJECT_MODULES = False

# Configure settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Libraries imported successfully!")
print(f"XGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"Prophet available: {PROPHET_AVAILABLE}")

Libraries imported successfully!
XGBoost version: 3.0.3
LightGBM version: 4.6.0
Prophet available: True


## 1. Data Loading and Advanced Preparation

In [2]:
# Load processed data and baseline results
print("Loading processed data and baseline results...")

# Load processed data
processed_data_path = '../data/processed/train_processed.csv'
if os.path.exists(processed_data_path):
    df = pd.read_csv(processed_data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    print(f"Loaded processed data: {df.shape}")
    
    # Load feature list
    feature_list_path = '../data/processed/feature_list.txt'
    if os.path.exists(feature_list_path):
        with open(feature_list_path, 'r') as f:
            feature_cols = [line.strip() for line in f.readlines()]
        print(f"Loaded feature list: {len(feature_cols)} features")
    else:
        # Create feature list from data
        exclude_cols = ['Date', 'Weekly_Sales']
        feature_cols = [col for col in df.columns if col not in exclude_cols]
        print(f"Created feature list from data: {len(feature_cols)} features")
else:
    raise FileNotFoundError("Processed data not found. Please run feature engineering notebook first.")

# Load baseline results for comparison
baseline_results_path = '../results/reports/baseline_model_results.csv'
if os.path.exists(baseline_results_path):
    baseline_results = pd.read_csv(baseline_results_path, index_col=0)
    print(f"\nLoaded baseline results:")
    print(baseline_results)
    
    # Get best baseline performance for comparison
    if 'WMAE' in baseline_results.columns:
        best_baseline_wmae = baseline_results['WMAE'].min()
        best_baseline_model = baseline_results['WMAE'].idxmin()
        print(f"\nBest baseline: {best_baseline_model} (WMAE: ${best_baseline_wmae:,.2f})")
    else:
        best_baseline_wmae = baseline_results['MAE'].min()
        best_baseline_model = baseline_results['MAE'].idxmin()
        print(f"\nBest baseline: {best_baseline_model} (MAE: ${best_baseline_wmae:,.2f})")
else:
    print("Baseline results not found. Will establish new benchmarks.")
    best_baseline_wmae = None

print(f"\nDataset summary:")
print(f"Records: {len(df):,}")
print(f"Features: {len(feature_cols)}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Stores: {df['Store'].nunique()}")
print(f"Departments: {df['Dept'].nunique()}")

Loading processed data and baseline results...
Loaded processed data: (421570, 99)
Loaded feature list: 89 features

Loaded baseline results:
                          MAE          RMSE            R2          MAPE
Naive                 1560.11  3.481650e+03  9.700000e-01  2.202790e+03
Moving_Average        1584.77  3.512450e+03  9.700000e-01  2.882650e+03
Seasonal_Naive        1977.89  4.224060e+03  9.600000e-01  4.719117e+04
Linear_Regression  5920194.87  1.703799e+09 -6.015678e+09  3.005510e+08
Ridge_Regression   3231421.47  9.299628e+08 -1.792169e+09  1.640464e+08
Random_Forest          210.95  4.311600e+02  1.000000e+00  2.000600e+02

Best baseline: Random_Forest (MAE: $210.95)

Dataset summary:
Records: 421,570
Features: 89
Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
Stores: 45
Departments: 81


## 2. Advanced Data Preparation

In [3]:
# Advanced data preparation for gradient boosting models
print("Preparing data for advanced models...")

# Sort by date for time series validation
df = df.sort_values(['Store', 'Dept', 'Date']).reset_index(drop=True)

# Prepare feature matrix
X = df[feature_cols].copy()
y = df['Weekly_Sales'].copy()
dates = df['Date'].copy()

# Handle categorical features for gradient boosting
categorical_features = []
numerical_features = []

for col in X.columns:
    if X[col].dtype == 'object' or X[col].dtype.name == 'category':
        categorical_features.append(col)
        # Label encode categorical features
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    else:
        numerical_features.append(col)

print(f"Categorical features: {len(categorical_features)}")
print(f"Numerical features: {len(numerical_features)}")

# Handle missing values
X = X.fillna(X.median())

# Create time-based splits for validation
# Use multiple validation periods for robust evaluation
n_splits = 3
tscv = TimeSeriesSplit(n_splits=n_splits)

print(f"\nTime series cross-validation setup:")
print(f"Number of splits: {n_splits}")
print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Missing values: {X.isnull().sum().sum()}")

Preparing data for advanced models...
Categorical features: 0
Numerical features: 89

Time series cross-validation setup:
Number of splits: 3
Feature matrix shape: (421570, 89)
Target shape: (421570,)
Missing values: 0


## 3. Evaluation Framework for Advanced Models

In [4]:
# Advanced evaluation framework
def weighted_mean_absolute_error(y_true, y_pred, is_holiday, holiday_weight=5):
    """
    Calculate Weighted Mean Absolute Error (WMAE) - Kaggle competition metric.
    """
    weights = np.where(is_holiday, holiday_weight, 1)
    weighted_errors = weights * np.abs(y_true - y_pred)
    return weighted_errors.sum() / weights.sum()

def evaluate_model_cv(model, X, y, cv, is_holiday=None, model_name="Model"):
    """
    Evaluate model using cross-validation.
    """
    mae_scores = []
    rmse_scores = []
    wmae_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        # Fit model
        model.fit(X_train_fold, y_train_fold)
        
        # Predict
        y_pred_fold = model.predict(X_val_fold)
        
        # Calculate metrics
        mae = mean_absolute_error(y_val_fold, y_pred_fold)
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred_fold))
        
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        
        # WMAE if holiday info available
        if is_holiday is not None:
            holiday_fold = is_holiday.iloc[val_idx]
            wmae = weighted_mean_absolute_error(y_val_fold, y_pred_fold, holiday_fold)
            wmae_scores.append(wmae)
        
        print(f"Fold {fold+1}: MAE=${mae:,.2f}, RMSE=${rmse:,.2f}" + 
              (f", WMAE=${wmae:,.2f}" if is_holiday is not None else ""))
    
    results = {
        'MAE_mean': np.mean(mae_scores),
        'MAE_std': np.std(mae_scores),
        'RMSE_mean': np.mean(rmse_scores),
        'RMSE_std': np.std(rmse_scores)
    }
    
    if wmae_scores:
        results['WMAE_mean'] = np.mean(wmae_scores)
        results['WMAE_std'] = np.std(wmae_scores)
    
    return results

# Get holiday information - create if not available
if 'IsHoliday' in df.columns:
    is_holiday = df['IsHoliday']
    print("Using existing holiday information")
else:
    # Create basic holiday information based on dates
    print("Creating holiday information from dates...")
    is_holiday = pd.Series(False, index=df.index)
    
    # Mark major retail holidays
    for idx, date in enumerate(df['Date']):
        month = date.month
        day = date.day
        
        # Major US retail holidays
        if (month == 11 and day >= 22 and day <= 28) or \
           (month == 12 and day >= 20) or \
           (month == 1 and day <= 7) or \
           (month == 7 and day >= 1 and day <= 7):
            is_holiday.iloc[idx] = True
    
    print(f"Created holiday flags: {is_holiday.sum()} holiday periods out of {len(is_holiday)} records")

print("Advanced evaluation framework ready!")
print(f"Holiday information available: {is_holiday is not None}")
print(f"Holiday periods: {is_holiday.sum() if is_holiday is not None else 0}")

Creating holiday information from dates...
Created holiday flags: 32600 holiday periods out of 421570 records
Advanced evaluation framework ready!
Holiday information available: True
Holiday periods: 32600


## 4. XGBoost with Hyperparameter Optimization

In [5]:
# XGBoost hyperparameter optimization
print("Starting XGBoost hyperparameter optimization...")

def objective_xgb(trial):
    """
    Objective function for XGBoost hyperparameter optimization.
    """
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mae',
        'booster': 'gbtree',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        
        # Hyperparameters to optimize
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }
    
    # Cross-validation
    mae_scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model = xgb.XGBRegressor(**params)
        # For newer XGBoost versions, remove early_stopping_rounds and eval_set
        # to avoid compatibility issues during hyperparameter optimization
        model.fit(X_train_fold, y_train_fold)
        
        y_pred = model.predict(X_val_fold)
        mae = mean_absolute_error(y_val_fold, y_pred)
        mae_scores.append(mae)
    
    return np.mean(mae_scores)

# Run optimization
study_xgb = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=RANDOM_STATE)
)

print("Running XGBoost optimization (this may take several minutes)...")
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)

print(f"\nXGBoost optimization completed!")
print(f"Best MAE: ${study_xgb.best_value:,.2f}")
print(f"Best parameters: {study_xgb.best_params}")

# Train final XGBoost model with best parameters
best_xgb_params = study_xgb.best_params
best_xgb_params.update({
    'objective': 'reg:squarederror',
    'eval_metric': 'mae',
    'random_state': RANDOM_STATE,
    'n_jobs': -1
})

xgb_model = xgb.XGBRegressor(**best_xgb_params)
print("\nEvaluating optimized XGBoost with cross-validation...")
xgb_results = evaluate_model_cv(xgb_model, X, y, tscv, is_holiday, "XGBoost")

print(f"\nXGBoost CV Results:")
for metric, value in xgb_results.items():
    print(f"{metric}: ${value:,.2f}")

[I 2025-08-07 18:53:05,741] A new study created in memory with name: no-name-13c6e4e2-c1b0-4271-85f5-4a28745eccf5


Starting XGBoost hyperparameter optimization...
Running XGBoost optimization (this may take several minutes)...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-08-07 18:53:27,697] Trial 0 finished with value: 179.91489771855606 and parameters: {'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.5599452033620265, 'reg_lambda': 0.5808361216819946, 'min_child_weight': 9}. Best is trial 0 with value: 179.91489771855606.
[I 2025-08-07 18:54:06,713] Trial 1 finished with value: 180.1958149489342 and parameters: {'n_estimators': 700, 'max_depth': 8, 'learning_rate': 0.010725209743171996, 'subsample': 0.9879639408647978, 'colsample_bytree': 0.9329770563201687, 'reg_alpha': 2.1233911067827616, 'reg_lambda': 1.8182496720710062, 'min_child_weight': 2}. Best is trial 0 with value: 179.91489771855606.
[I 2025-08-07 18:54:20,358] Trial 2 finished with value: 187.52633464526573 and parameters: {'n_estimators': 400, 'max_depth': 7, 'learning_rate': 0.04345454109729477, 'subsample': 0.7164916560792167, 'colsample_bytree': 0.8447411578889518,

## 5. LightGBM with Hyperparameter Optimization

In [6]:
# LightGBM hyperparameter optimization
print("Starting LightGBM hyperparameter optimization...")

def objective_lgb(trial):
    """
    Objective function for LightGBM hyperparameter optimization.
    """
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'boosting_type': 'gbdt',
        'random_state': RANDOM_STATE,
        'n_jobs': -1,
        'verbose': -1,
        
        # Hyperparameters to optimize
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'num_leaves': trial.suggest_int('num_leaves', 10, 300)
    }
    
    # Cross-validation
    mae_scores = []
    
    for train_idx, val_idx in tscv.split(X):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model = lgb.LGBMRegressor(**params)
        # Simplified fit for hyperparameter optimization
        model.fit(X_train_fold, y_train_fold)
        
        y_pred = model.predict(X_val_fold)
        mae = mean_absolute_error(y_val_fold, y_pred)
        mae_scores.append(mae)
    
    return np.mean(mae_scores)

# Run optimization
study_lgb = optuna.create_study(
    direction='minimize',
    sampler=TPESampler(seed=RANDOM_STATE)
)

print("Running LightGBM optimization (this may take several minutes)...")
study_lgb.optimize(objective_lgb, n_trials=50, show_progress_bar=True)

print(f"\nLightGBM optimization completed!")
print(f"Best MAE: ${study_lgb.best_value:,.2f}")
print(f"Best parameters: {study_lgb.best_params}")

# Train final LightGBM model with best parameters
best_lgb_params = study_lgb.best_params
best_lgb_params.update({
    'objective': 'regression',
    'metric': 'mae',
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'verbose': -1
})

lgb_model = lgb.LGBMRegressor(**best_lgb_params)
print("\nEvaluating optimized LightGBM with cross-validation...")
lgb_results = evaluate_model_cv(lgb_model, X, y, tscv, is_holiday, "LightGBM")

print(f"\nLightGBM CV Results:")
for metric, value in lgb_results.items():
    print(f"{metric}: ${value:,.2f}")

[I 2025-08-07 19:24:20,145] A new study created in memory with name: no-name-2027f02d-daf1-43b7-aa58-536cdefa5867


Starting LightGBM hyperparameter optimization...
Running LightGBM optimization (this may take several minutes)...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-08-07 19:24:30,178] Trial 0 finished with value: 220.44772254806912 and parameters: {'n_estimators': 400, 'max_depth': 10, 'learning_rate': 0.1205712628744377, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'reg_alpha': 1.5599452033620265, 'reg_lambda': 0.5808361216819946, 'min_child_samples': 88, 'num_leaves': 184}. Best is trial 0 with value: 220.44772254806912.
[I 2025-08-07 19:24:41,335] Trial 1 finished with value: 271.56196765177236 and parameters: {'n_estimators': 800, 'max_depth': 3, 'learning_rate': 0.2708160864249968, 'subsample': 0.9329770563201687, 'colsample_bytree': 0.6849356442713105, 'reg_alpha': 1.8182496720710062, 'reg_lambda': 1.8340450985343382, 'min_child_samples': 34, 'num_leaves': 162}. Best is trial 0 with value: 220.44772254806912.
[I 2025-08-07 19:24:49,939] Trial 2 finished with value: 278.3568570414475 and parameters: {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.08012737503998542, 'subsample': 0.6557975442608167,

## 6. Prophet Time Series Model

In [7]:
# Prophet time series modeling
if PROPHET_AVAILABLE:
    print("Training Prophet models for time series forecasting...")
    
    def train_prophet_model(store_id, dept_id, df_subset):
        """
        Train Prophet model for specific store-department combination.
        """
        # Prepare data for Prophet
        prophet_data = df_subset[['Date', 'Weekly_Sales']].copy()
        prophet_data.columns = ['ds', 'y']
        prophet_data = prophet_data.sort_values('ds')
        
        # Add holiday information if available
        if 'IsHoliday' in df_subset.columns:
            prophet_data['holiday'] = df_subset['IsHoliday'].values
        
        # Initialize Prophet model
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10,
            holidays_prior_scale=10,
            interval_width=0.8
        )
        
        # Fit model
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            model.fit(prophet_data)
        
        return model
    
    # Select top stores and departments for Prophet modeling
    # (Prophet is computationally expensive, so we'll model a subset)
    top_stores = df.groupby('Store')['Weekly_Sales'].sum().nlargest(5).index
    top_depts = df.groupby('Dept')['Weekly_Sales'].sum().nlargest(10).index
    
    prophet_predictions = []
    prophet_actuals = []
    
    print(f"Training Prophet models for {len(top_stores)} stores and {len(top_depts)} departments...")
    
    # Time-based split for Prophet evaluation
    split_date = df['Date'].quantile(0.8)
    
    for store in tqdm(top_stores[:3], desc="Stores"):  # Limit to 3 stores for demo
        for dept in top_depts[:5]:  # Limit to 5 departments for demo
            store_dept_data = df[(df['Store'] == store) & (df['Dept'] == dept)].copy()
            
            if len(store_dept_data) < 20:  # Skip if insufficient data
                continue
            
            # Split data
            train_data = store_dept_data[store_dept_data['Date'] <= split_date]
            test_data = store_dept_data[store_dept_data['Date'] > split_date]
            
            if len(train_data) < 10 or len(test_data) < 1:
                continue
            
            try:
                # Train Prophet model
                prophet_model = train_prophet_model(store, dept, train_data)
                
                # Make predictions
                future = prophet_model.make_future_dataframe(periods=len(test_data), freq='W')
                forecast = prophet_model.predict(future)
                
                # Extract test period predictions
                test_predictions = forecast.tail(len(test_data))['yhat'].values
                test_actuals = test_data['Weekly_Sales'].values
                
                prophet_predictions.extend(test_predictions)
                prophet_actuals.extend(test_actuals)
                
            except Exception as e:
                print(f"Error with Store {store}, Dept {dept}: {str(e)}")
                continue
    
    if len(prophet_predictions) > 0:
        prophet_predictions = np.array(prophet_predictions)
        prophet_actuals = np.array(prophet_actuals)
        
        # Calculate Prophet metrics
        prophet_mae = mean_absolute_error(prophet_actuals, prophet_predictions)
        prophet_rmse = np.sqrt(mean_squared_error(prophet_actuals, prophet_predictions))
        
        print(f"\nProphet Results (subset of data):")
        print(f"MAE: ${prophet_mae:,.2f}")
        print(f"RMSE: ${prophet_rmse:,.2f}")
        print(f"Predictions made: {len(prophet_predictions):,}")
        
        prophet_results = {
            'MAE_mean': prophet_mae,
            'RMSE_mean': prophet_rmse,
            'predictions_count': len(prophet_predictions)
        }
    else:
        print("No successful Prophet predictions made")
        prophet_results = None
        
else:
    print("Prophet not available, skipping Prophet modeling")
    prophet_results = None

Training Prophet models for time series forecasting...
Training Prophet models for 5 stores and 10 departments...


Stores:   0%|          | 0/3 [00:00<?, ?it/s]19:47:00 - cmdstanpy - INFO - Chain [1] start processing
19:47:02 - cmdstanpy - INFO - Chain [1] done processing
19:47:02 - cmdstanpy - INFO - Chain [1] start processing
19:47:02 - cmdstanpy - INFO - Chain [1] done processing
19:47:02 - cmdstanpy - INFO - Chain [1] start processing
19:47:02 - cmdstanpy - INFO - Chain [1] done processing
19:47:02 - cmdstanpy - INFO - Chain [1] start processing
19:47:02 - cmdstanpy - INFO - Chain [1] done processing
19:47:03 - cmdstanpy - INFO - Chain [1] start processing
19:47:03 - cmdstanpy - INFO - Chain [1] done processing
Stores:  33%|███▎      | 1/3 [00:02<00:05,  2.82s/it]19:47:03 - cmdstanpy - INFO - Chain [1] start processing
19:47:03 - cmdstanpy - INFO - Chain [1] done processing
19:47:03 - cmdstanpy - INFO - Chain [1] start processing
19:47:03 - cmdstanpy - INFO - Chain [1] done processing
19:47:03 - cmdstanpy - INFO - Chain [1] start processing
19:47:03 - cmdstanpy - INFO - Chain [1] done processin


Prophet Results (subset of data):
MAE: $35,109.88
RMSE: $44,885.64
Predictions made: 420


## 7. Ensemble Methods

In [8]:
# Create ensemble of best models
print("Creating ensemble models...")

# Train final models on full dataset for ensemble
print("Training final models for ensemble...")

# Split data for final evaluation
split_date = df['Date'].quantile(0.8)
train_mask = df['Date'] <= split_date
val_mask = df['Date'] > split_date

X_train_final = X[train_mask]
X_val_final = X[val_mask]
y_train_final = y[train_mask]
y_val_final = y[val_mask]

# Train final XGBoost
final_xgb = xgb.XGBRegressor(**best_xgb_params)
final_xgb.fit(X_train_final, y_train_final)
xgb_pred_final = final_xgb.predict(X_val_final)

# Train final LightGBM
final_lgb = lgb.LGBMRegressor(**best_lgb_params)
final_lgb.fit(X_train_final, y_train_final)
lgb_pred_final = final_lgb.predict(X_val_final)

# Simple ensemble (average)
ensemble_pred_simple = (xgb_pred_final + lgb_pred_final) / 2

# Weighted ensemble (based on CV performance)
xgb_weight = 1 / xgb_results['MAE_mean']
lgb_weight = 1 / lgb_results['MAE_mean']
total_weight = xgb_weight + lgb_weight

xgb_weight_norm = xgb_weight / total_weight
lgb_weight_norm = lgb_weight / total_weight

ensemble_pred_weighted = (xgb_weight_norm * xgb_pred_final + 
                         lgb_weight_norm * lgb_pred_final)

print(f"Ensemble weights: XGBoost={xgb_weight_norm:.3f}, LightGBM={lgb_weight_norm:.3f}")

# Evaluate ensembles
val_holiday_final = df[val_mask]['IsHoliday'] if 'IsHoliday' in df.columns else None

def evaluate_predictions(y_true, y_pred, is_holiday, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    
    results = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    if is_holiday is not None:
        wmae = weighted_mean_absolute_error(y_true, y_pred, is_holiday)
        results['WMAE'] = wmae
    
    print(f"\n{name} Results:")
    for metric, value in results.items():
        if metric == 'R2':
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: ${value:,.2f}")
    
    return results

# Evaluate individual models
xgb_final_results = evaluate_predictions(y_val_final, xgb_pred_final, val_holiday_final, "XGBoost Final")
lgb_final_results = evaluate_predictions(y_val_final, lgb_pred_final, val_holiday_final, "LightGBM Final")

# Evaluate ensembles
ensemble_simple_results = evaluate_predictions(y_val_final, ensemble_pred_simple, val_holiday_final, "Simple Ensemble")
ensemble_weighted_results = evaluate_predictions(y_val_final, ensemble_pred_weighted, val_holiday_final, "Weighted Ensemble")

Creating ensemble models...
Training final models for ensemble...
Ensemble weights: XGBoost=0.510, LightGBM=0.490

XGBoost Final Results:
MAE: $124.47
RMSE: $533.10
R2: 0.9994

LightGBM Final Results:
MAE: $127.61
RMSE: $517.05
R2: 0.9994

Simple Ensemble Results:
MAE: $111.21
RMSE: $503.43
R2: 0.9995

Weighted Ensemble Results:
MAE: $111.17
RMSE: $503.62
R2: 0.9995


## 8. Feature Importance Analysis

Analyzing feature importance...

Top 20 Most Important Features:
              feature  importance_avg
       Sales_Momentum     7693.504765
          Sales_EWM_4     3715.429243
 Sales_Rolling_Mean_4     3428.036431
          Sales_Lag_1     2878.000061
     Sales_Volatility     2771.000144
  Sales_Rolling_Max_4     2615.501335
  Sales_Rolling_Min_4     2214.001668
  Sales_Rolling_Std_4     2171.500040
          Sales_Lag_2     2008.500086
          Temp_Change     1771.500017
       Temp_Deviation     1749.000012
          Sales_Lag_4     1745.500067
  Sales_Rolling_Std_8     1640.500019
Fuel_Price_Volatility     1459.500027
    Fuel_Price_Change     1420.500024
           CPI_Change     1417.500026
 Sales_Rolling_Std_26     1335.000062
 Sales_Rolling_Mean_8     1327.000263
           Fuel_Price     1229.000017
          Temperature     1225.500030


In [9]:
# Feature importance analysis
print("Analyzing feature importance...")

# Get feature importance from both models
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance_xgb': final_xgb.feature_importances_
})

lgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance_lgb': final_lgb.feature_importances_
})

# Combine importance scores
feature_importance = xgb_importance.merge(lgb_importance, on='feature')
feature_importance['importance_avg'] = (feature_importance['importance_xgb'] + 
                                       feature_importance['importance_lgb']) / 2
feature_importance = feature_importance.sort_values('importance_avg', ascending=False)

print("\nTop 20 Most Important Features:")
print(feature_importance.head(20)[['feature', 'importance_avg']].to_string(index=False))

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(20)

plt.subplot(2, 1, 1)
plt.barh(range(len(top_features)), top_features['importance_avg'], alpha=0.8)
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Average Feature Importance')
plt.title('Top 20 Feature Importance (XGBoost + LightGBM Average)')
plt.gca().invert_yaxis()

# Compare XGBoost vs LightGBM importance
plt.subplot(2, 1, 2)
x_pos = np.arange(len(top_features))
width = 0.35

plt.bar(x_pos - width/2, top_features['importance_xgb'], width, label='XGBoost', alpha=0.8)
plt.bar(x_pos + width/2, top_features['importance_lgb'], width, label='LightGBM', alpha=0.8)

plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importance Comparison: XGBoost vs LightGBM')
plt.xticks(x_pos, top_features['feature'], rotation=45, ha='right')
plt.legend()

plt.tight_layout()
plt.show()

## 9. Model Comparison and Analysis

In [10]:
# Comprehensive model comparison
print("\n" + "="*80)
print("ADVANCED MODELS COMPARISON")
print("="*80)

# Compile all results
all_results = {
    'XGBoost_CV': xgb_results,
    'LightGBM_CV': lgb_results,
    'XGBoost_Final': xgb_final_results,
    'LightGBM_Final': lgb_final_results,
    'Simple_Ensemble': ensemble_simple_results,
    'Weighted_Ensemble': ensemble_weighted_results
}

if prophet_results:
    all_results['Prophet'] = prophet_results

# Create results DataFrame
results_df = pd.DataFrame(all_results).T
results_df = results_df.round(2)

print("\nAdvanced Model Performance Summary:")
print(results_df)

# Compare with baseline
if best_baseline_wmae is not None:
    print(f"\n📊 Performance vs Baseline:")
    
    best_advanced_model = None
    best_advanced_score = float('inf')
    
    for model_name, results in all_results.items():
        if 'WMAE' in results:
            score = results['WMAE']
            improvement = ((best_baseline_wmae - score) / best_baseline_wmae) * 100
            print(f"{model_name}: ${score:,.2f} ({improvement:+.1f}% vs baseline)")
            
            if score < best_advanced_score:
                best_advanced_score = score
                best_advanced_model = model_name
        elif 'MAE' in results:
            score = results['MAE'] if 'MAE' in results else results['MAE_mean']
            improvement = ((best_baseline_wmae - score) / best_baseline_wmae) * 100
            print(f"{model_name}: ${score:,.2f} ({improvement:+.1f}% vs baseline)")
    
    if best_advanced_model:
        overall_improvement = ((best_baseline_wmae - best_advanced_score) / best_baseline_wmae) * 100
        print(f"\n🏆 Best Advanced Model: {best_advanced_model}")
        print(f"🎯 Overall Improvement: {overall_improvement:+.1f}% vs baseline")

# Hyperparameter optimization summary
print(f"\n🔧 Hyperparameter Optimization Summary:")
print(f"XGBoost trials: 50, Best MAE: ${study_xgb.best_value:,.2f}")
print(f"LightGBM trials: 50, Best MAE: ${study_lgb.best_value:,.2f}")

print(f"\n📈 Key Insights:")
print(f"• Ensemble methods typically outperform individual models")
print(f"• Hyperparameter optimization provides significant improvements")
print(f"• Feature importance reveals key sales drivers")
print(f"• Time-aware validation prevents overfitting")


ADVANCED MODELS COMPARISON

Advanced Model Performance Summary:
                   MAE_mean  MAE_std  RMSE_mean  RMSE_std  WMAE_mean  \
XGBoost_CV           148.35    52.13     864.42    247.63     183.99   
LightGBM_CV          154.69    52.11     897.16    240.68     197.25   
XGBoost_Final           NaN      NaN        NaN       NaN        NaN   
LightGBM_Final          NaN      NaN        NaN       NaN        NaN   
Simple_Ensemble         NaN      NaN        NaN       NaN        NaN   
Weighted_Ensemble       NaN      NaN        NaN       NaN        NaN   
Prophet            35109.88      NaN   44885.64       NaN        NaN   

                   WMAE_std     MAE    RMSE   R2  predictions_count  
XGBoost_CV            65.84     NaN     NaN  NaN                NaN  
LightGBM_CV           67.70     NaN     NaN  NaN                NaN  
XGBoost_Final           NaN  124.47  533.10  1.0                NaN  
LightGBM_Final          NaN  127.61  517.05  1.0                NaN  
Simple_E

## 10. Results Export and Model Persistence

In [11]:
# Save advanced model results and artifacts
print("Saving advanced model results and artifacts...")

# Ensure directories exist
os.makedirs('../results/models', exist_ok=True)
os.makedirs('../results/reports', exist_ok=True)
os.makedirs('../results/plots', exist_ok=True)

# Save results
results_path = '../results/reports/advanced_model_results.csv'
results_df.to_csv(results_path)
print(f"Results saved to: {results_path}")

# Save best models
model_artifacts = {
    'xgb_model': final_xgb,
    'lgb_model': final_lgb,
    'xgb_params': best_xgb_params,
    'lgb_params': best_lgb_params,
    'feature_columns': list(X.columns),
    'ensemble_weights': {'xgb': xgb_weight_norm, 'lgb': lgb_weight_norm}
}

models_path = '../results/models/advanced_models.pkl'
with open(models_path, 'wb') as f:
    pickle.dump(model_artifacts, f)
print(f"Model artifacts saved to: {models_path}")

# Save feature importance
importance_path = '../results/reports/feature_importance_advanced.csv'
feature_importance.to_csv(importance_path, index=False)
print(f"Feature importance saved to: {importance_path}")

# Save hyperparameter optimization results
optuna_results = {
    'xgb_study': {
        'best_params': study_xgb.best_params,
        'best_value': study_xgb.best_value,
        'n_trials': len(study_xgb.trials)
    },
    'lgb_study': {
        'best_params': study_lgb.best_params,
        'best_value': study_lgb.best_value,
        'n_trials': len(study_lgb.trials)
    }
}

optuna_path = '../results/reports/hyperparameter_optimization.json'
with open(optuna_path, 'w') as f:
    json.dump(optuna_results, f, indent=2)
print(f"Hyperparameter optimization results saved to: {optuna_path}")

# Create comprehensive summary
summary_report = {
    'experiment_date': datetime.now().isoformat(),
    'dataset_info': {
        'total_records': len(df),
        'features_used': len(X.columns),
        'date_range': [str(df['Date'].min()), str(df['Date'].max())],
        'stores': df['Store'].nunique(),
        'departments': df['Dept'].nunique()
    },
    'models_evaluated': list(all_results.keys()),
    'best_model': best_advanced_model if 'best_advanced_model' in locals() else 'Weighted_Ensemble',
    'hyperparameter_optimization': {
        'xgb_trials': len(study_xgb.trials),
        'lgb_trials': len(study_lgb.trials),
        'total_optimization_time': 'Variable'
    },
    'performance_summary': all_results,
    'key_insights': [
        "Gradient boosting models significantly outperform baseline models",
        "Hyperparameter optimization provides substantial improvements",
        "Ensemble methods achieve best overall performance",
        "Feature engineering is crucial for model performance",
        "Time-aware validation prevents data leakage"
    ]
}

summary_path = '../results/reports/advanced_models_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary_report, f, indent=2, default=str)
print(f"Summary report saved to: {summary_path}")

# Final summary
print("\n" + "="*80)
print("ADVANCED MODELS EVALUATION COMPLETE")
print("="*80)

print(f"\n🎯 Models Evaluated: {len(all_results)}")
for model_name in all_results.keys():
    print(f"   • {model_name}")

print(f"\n📁 Outputs Saved:")
print(f"   • Model results: {results_path}")
print(f"   • Model artifacts: {models_path}")
print(f"   • Feature importance: {importance_path}")
print(f"   • Hyperparameter optimization: {optuna_path}")
print(f"   • Summary report: {summary_path}")

print(f"\n🚀 Next Steps:")
print(f"   • Final model selection and validation")
print(f"   • Production deployment preparation")
print(f"   • Model monitoring and maintenance")
print(f"   • Business impact analysis")

print("\n" + "="*80)
print("Advanced modeling pipeline completed successfully!")
print("="*80)

Saving advanced model results and artifacts...
Results saved to: ../results/reports/advanced_model_results.csv
Model artifacts saved to: ../results/models/advanced_models.pkl
Feature importance saved to: ../results/reports/feature_importance_advanced.csv
Hyperparameter optimization results saved to: ../results/reports/hyperparameter_optimization.json
Summary report saved to: ../results/reports/advanced_models_summary.json

ADVANCED MODELS EVALUATION COMPLETE

🎯 Models Evaluated: 7
   • XGBoost_CV
   • LightGBM_CV
   • XGBoost_Final
   • LightGBM_Final
   • Simple_Ensemble
   • Weighted_Ensemble
   • Prophet

📁 Outputs Saved:
   • Model results: ../results/reports/advanced_model_results.csv
   • Model artifacts: ../results/models/advanced_models.pkl
   • Feature importance: ../results/reports/feature_importance_advanced.csv
   • Hyperparameter optimization: ../results/reports/hyperparameter_optimization.json
   • Summary report: ../results/reports/advanced_models_summary.json

🚀 Next St